In [9]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'fourier'
CHANNEL = 'blue'

In [10]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_fourier_full_pastis.ipynb


In [11]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [12]:
from scipy import spatial
from sklearn.decomposition import PCA

In [13]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [14]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [15]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [16]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [17]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10,11,...,24,25,26,27,28,29,30,31,32,33
2,14.56,-0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,...,0.0,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.0,0.0,-0.0
3,-0.00,11.23,0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,...,-0.0,-0.00,-0.00,-0.00,-0.00,0.00,0.00,0.0,-0.0,-0.0
4,0.00,0.00,9.62,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,...,0.0,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.0,0.0,0.0
5,-0.00,0.00,0.00,7.68,0.00,-0.00,0.00,0.00,0.00,0.00,...,0.0,0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.0,-0.0,0.0
6,-0.00,0.00,-0.00,0.00,6.22,0.00,0.00,-0.00,-0.00,-0.00,...,-0.0,0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.0,0.0,0.0
7,0.00,0.00,-0.00,-0.00,0.00,4.93,0.00,-0.00,0.00,0.00,...,-0.0,-0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.0,-0.0,0.0
8,0.00,-0.00,0.00,0.00,0.00,0.00,4.07,0.00,0.00,-0.00,...,-0.0,0.00,0.00,0.00,-0.00,-0.00,-0.00,0.0,-0.0,-0.0
9,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,3.36,-0.00,0.00,...,0.0,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.0,0.0,-0.0
10,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,2.75,0.00,...,0.0,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.0,0.0,0.0
11,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,2.24,...,0.0,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.0,-0.0,-0.0


In [18]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10,11,...,24,25,26,27,28,29,30,31,32,33
2,1.00000,-0.00007,0.00002,-0.00003,-0.00002,0.00004,0.00003,-0.00004,0.00003,-0.00000,...,0.00004,0.00003,0.00004,-0.00005,-0.00000,-0.00001,0.00006,0.00003,0.00007,-0.00001
3,-0.00007,1.00000,0.00000,0.00002,0.00003,0.00005,-0.00002,-0.00003,0.00004,-0.00000,...,-0.00003,-0.00004,-0.00004,-0.00002,-0.00001,0.00000,0.00001,0.00003,-0.00002,-0.00006
4,0.00002,0.00000,1.00000,0.00007,-0.00003,-0.00001,0.00002,-0.00001,0.00002,0.00000,...,0.00001,-0.00001,-0.00001,-0.00003,0.00003,-0.00003,-0.00000,0.00004,0.00001,0.00005
5,-0.00003,0.00002,0.00007,1.00000,0.00004,-0.00002,0.00002,0.00005,0.00001,0.00000,...,0.00002,0.00003,-0.00001,-0.00003,-0.00003,0.00003,-0.00000,0.00004,-0.00001,0.00001
6,-0.00002,0.00003,-0.00003,0.00004,1.00000,0.00000,0.00003,-0.00001,-0.00002,-0.00000,...,-0.00000,0.00001,-0.00000,-0.00002,0.00001,-0.00001,-0.00000,0.00002,0.00002,0.00001
7,0.00004,0.00005,-0.00001,-0.00002,0.00000,1.00000,0.00002,-0.00000,0.00001,0.00000,...,-0.00006,-0.00001,-0.00000,-0.00001,0.00004,0.00002,-0.00007,-0.00005,-0.00003,0.00008
8,0.00003,-0.00002,0.00002,0.00002,0.00003,0.00002,1.00000,0.00007,0.00004,-0.00000,...,-0.00005,0.00000,0.00002,0.00000,-0.00007,-0.00002,-0.00000,0.00001,-0.00004,-0.00003
9,-0.00004,-0.00003,-0.00001,0.00005,-0.00001,-0.00000,0.00007,1.00000,-0.00007,0.00001,...,0.00001,-0.00007,-0.00001,-0.00001,-0.00005,-0.00003,-0.00001,0.00001,0.00003,-0.00001
10,0.00003,0.00004,0.00002,0.00001,-0.00002,0.00001,0.00004,-0.00007,1.00000,0.00005,...,0.00001,-0.00001,0.00001,-0.00006,0.00002,-0.00001,0.00002,-0.00001,0.00001,0.00001
11,-0.00000,-0.00000,0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00001,0.00005,1.00000,...,0.00002,-0.00003,0.00001,-0.00001,-0.00003,0.00002,-0.00005,0.00001,-0.00003,-0.00001


In [19]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.002419570422242773

In [20]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[6.64951488e+00 3.95289529e+00 2.88844557e+00 1.84065850e+00
 1.20450535e+00 7.55286084e-01 5.15372958e-01 3.49573268e-01
 2.34265368e-01 1.55476826e-01 1.02445835e-01 7.36790592e-02
 4.26751042e-02 3.00509812e-02 1.91820442e-02 1.21035339e-02
 7.51390456e-03 4.58465385e-03 2.73257268e-03 1.71844720e-03
 9.67153636e-04 5.35551464e-04 2.91849818e-04 1.60950421e-04
 8.04061381e-05 3.89744213e-05 1.89233936e-05 8.17114158e-06
 1.06237319e-06 2.51109720e-07 4.86392865e-08 1.31015950e-30]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.997202,-0.057863,-0.034695,-0.021461,-0.015257,-0.011073,-0.008785,-0.007089,-0.005667,-0.004575,...,-0.000195,-0.000147,-0.000103,-0.000077,-0.000053,-0.000036,-0.000012,-0.000006,-0.000002,-0.000002
1,0.052307,0.991952,-0.102811,-0.038698,-0.023698,-0.015980,-0.012332,-0.009693,-0.007637,-0.006115,...,-0.000264,-0.000199,-0.000141,-0.000098,-0.000070,-0.000047,-0.000017,-0.000008,-0.000004,-0.000003
2,0.037060,0.095758,0.990216,-0.078273,-0.038244,-0.023556,-0.017413,-0.013408,-0.010461,-0.008286,...,-0.000348,-0.000262,-0.000187,-0.000132,-0.000090,-0.000065,-0.000023,-0.000011,-0.000005,-0.000002
3,0.023787,0.041047,0.068206,0.991390,-0.085376,-0.038000,-0.025412,-0.018479,-0.014042,-0.010897,...,-0.000440,-0.000328,-0.000237,-0.000168,-0.000119,-0.000079,-0.000029,-0.000014,-0.000007,-0.000003
4,0.017898,0.027064,0.037392,0.075801,0.990848,-0.080728,-0.042020,-0.027643,-0.019836,-0.014888,...,-0.000573,-0.000428,-0.000306,-0.000216,-0.000151,-0.000105,-0.000037,-0.000019,-0.000009,-0.000005
5,0.013081,0.018497,0.023641,0.036686,0.069164,0.989149,-0.101247,-0.047002,-0.029476,-0.020720,...,-0.000733,-0.000543,-0.000387,-0.000272,-0.000188,-0.000131,-0.000050,-0.000026,-0.000012,-0.000005
6,0.011015,0.015170,0.018658,0.026519,0.040663,0.088325,0.986671,-0.105808,-0.048541,-0.030486,...,-0.000940,-0.000695,-0.000494,-0.000347,-0.000251,-0.000170,-0.000061,-0.000031,-0.000016,-0.000008
7,0.009324,0.012545,0.015136,0.020433,0.028723,0.047170,0.090671,0.985584,-0.108632,-0.049646,...,-0.001187,-0.000899,-0.000633,-0.000444,-0.000318,-0.000218,-0.000078,-0.000039,-0.000018,-0.000010
8,0.007801,0.010360,0.012384,0.016305,0.021782,0.031838,0.047875,0.092342,0.984850,-0.110231,...,-0.001515,-0.001135,-0.000806,-0.000573,-0.000399,-0.000276,-0.000098,-0.000051,-0.000024,-0.000012
9,0.006529,0.008603,0.010186,0.013169,0.017074,0.023538,0.032141,0.048347,0.092755,0.983950,...,-0.001928,-0.001451,-0.001026,-0.000723,-0.000514,-0.000349,-0.000128,-0.000064,-0.000031,-0.000016


In [21]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0027978775897574693,
 0.008048077922727437,
 0.009784266004697617,
 0.008609595687119054,
 0.009151778238176034,
 0.010850516885189121,
 0.013329059899180873,
 0.014416343944080312,
 0.015149562733897115,
 0.01605046178850722,
 0.020748573664037817,
 0.020771349212181378,
 0.02017498191346656,
 0.024376696486629656,
 0.02227388665875385,
 0.023046795013943577,
 0.02412506444992335,
 0.02538246403773481,
 0.029892196893062106,
 0.03236993999892268,
 0.030408566988811114,
 0.033087445126466,
 0.038714036717129696,
 0.04289231340024324,
 0.04497004986987663,
 0.055136659395647825,
 0.0697950508664642,
 0.055321437059460576,
 0.03512074695470446,
 0.06660611409581241,
 0.14980369370837465,
 0.13215491395821033]